In [1]:
import os
import requests
from  openai import OpenAI
import gradio as gr
import speech_recognition as sr
import json
from dotenv import load_dotenv

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
weather_api_key = os.getenv('WEATHER_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if weather_api_key:
    print("weather API Key exists")
else:
    print("weather API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-
weather API Key exists


In [3]:
system_message = "You are a helpful assistant for weather. "
system_message += "You need to fetch the current, historical and forecast the weather data using weather api and provide the response"

In [4]:
def fetch_current_weather(location):
    url = f"http://api.weatherapi.com/v1/current.json?key={weather_api_key}&q={location}&aqi=yes"
    return requests.get(url).json()

In [5]:
def fetch_forecast_weather(location, days=3):
    url = f"http://api.weatherapi.com/v1/forecast.json?key={weather_api_key}&q={location}&days={days}&aqi=yes&alerts=yes"
    return requests.get(url).json()

In [6]:
def fetch_historical_weather(location, date):
    url = f"http://api.weatherapi.com/v1/history.json?key={weather_api_key}&q={location}&dt={date}&aqi=yes"
    return requests.get(url).json()

In [7]:
# Weather function used as a tool by OpenAI
def get_weatherapi_data(location, mode="current", date=None, forecast_days=3):
    if mode == "current":
        return fetch_current_weather(location)
    elif mode == "forecast":
        return fetch_forecast_weather(location, days=forecast_days)
    elif mode == "historical":
        if not date:
            # Default: yesterday
            date = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        return fetch_historical_weather(location, date)
    else:
        return {"error": "Unknown mode."}

In [8]:
# Tool schema for OpenAI tool-calling
weatherapi_tool_schema = [
  {
    "type": "function",
    "function": {
      "name": "get_weatherapi_data",
      "description": "Fetches current, forecast, or historical weather data from WeatherAPI.com for a given location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "Name of the city, region, or coordinates."
          },
          "mode": {
            "type": "string",
            "enum": ["current", "forecast", "historical"],
            "description": "Type of weather data required."
          },
          "date": {
            "type": "string",
            "description": "Date for historical data in YYYY-MM-DD format. Only needed if mode is 'historical'."
          },
          "forecast_days": {
            "type": "integer",
            "description": "Number of forecast days (1-10). Only needed if mode is 'forecast'."
          }
        },
        "required": ["location", "mode"]
      }
    }
  }
]

In [9]:
def audio_to_text(audio_filepath):
    if audio_filepath is None or audio_filepath == "":
        return ""
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_filepath) as source:
            audio = recognizer.record(source)
        try:
            transcript = recognizer.recognize_google(audio)
            return transcript
        except sr.UnknownValueError:
            return ""
        except sr.RequestError as e:
            return f"Speech recognition service error: {e}"
    except Exception as e:
        return f"Error opening audio file: {str(e)}"

In [10]:
def chat_agent(city, mode, date, forecast_days, audio=None):
    user_query = city
    if audio:
        spoken_text = audio_to_text(audio)
    print("Recognized speech:", spoken_text)
    if spoken_text and spoken_text.strip().lower() != "flic en flac":
        user_query = spoken_text
    else:
        if not city.strip():
            return "Sorry, I could not recognize your speech. Please try again or type your city."

    if not user_query.strip():
        return "Please provide a location by text or speech."

    # Compose tool function arguments as the LLM would
    args = {
        "location": user_query,
        "mode": mode
    }
    if mode == "historical" and date:
        args["date"] = date
    if mode == "forecast":
        try:
            n_days = int(forecast_days)
        except:
            n_days = 3
        args["forecast_days"] = n_days

    openai.api_key = openai_api_key

    # LLM call for tool use
    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=[{"role": "user", "content": f"Get me {mode} weather for {user_query}"+(f' on {date}' if date and mode=="historical" else "")+(f' for {forecast_days} days' if forecast_days and mode=="forecast" else "")}],
        tools=weatherapi_tool_schema,
        tool_choice={"type": "function", "function": {"name": "get_weatherapi_data", "arguments": json.dumps(args)}}
    )
    message = response.choices[0].message

    if hasattr(message, "tool_calls") and message.tool_calls:
        tool_call = message.tool_calls[0]
        args2 = json.loads(tool_call.function.arguments)  # not really needed, already have args
        location = args2.get("location", user_query)
        mode = args2.get("mode", mode)
        date = args2.get("date", date)
        forecast_days = args2.get("forecast_days", forecast_days)
        weather_data = get_weatherapi_data(location, mode, date, forecast_days)
        tool_result = f"Weather data (mode={mode}) for {location}:\n{json.dumps(weather_data, indent=2)[:3000]}"
        followup = openai.chat.completions.create(
            model="gpt-4-0613",
            messages=[
                {"role": "user", "content": f"Get me {mode} weather for {location}"},
                message,
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result
                }
            ]
        )
        answer = followup.choices[0].message.content.strip()
        return answer
    else:
        return getattr(message, "content", "")

In [11]:
def update_date_visibility(mode):
    return gr.update(visible=(mode=="historical"))

def update_days_visibility(mode):
    return gr.update(visible=(mode=="forecast"))

In [13]:
with gr.Blocks() as demo:
    gr.Markdown("## Weather Chat Agent (Current, Historical, Forecast)")

    with gr.Row():
        city_input = gr.Textbox(label="City/Location")
        mode_input = gr.Dropdown(
            ["current", "historical", "forecast"],
            value="current",
            label="Weather Mode")
    with gr.Row():
        date_input = gr.Textbox(label="Date for historical (YYYY-MM-DD)", visible=False)
        days_input = gr.Textbox(label="Forecast Days (for forecast)", value="3", visible=False)
    audio_input = gr.Audio(type="filepath", format="wav", label="Or Speak your City/Location (optional)")
    output_box = gr.Textbox(label="Weather Info", lines=8)
    btn = gr.Button("Get Weather")

    # Show/hide date and days inputs based on dropdown
    mode_input.change(update_date_visibility, mode_input, date_input)
    mode_input.change(update_days_visibility, mode_input, days_input)
    btn.click(
        chat_agent,
        [city_input, mode_input, date_input, days_input, audio_input],
        output_box
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Recognized speech: Error opening audio file: FLAC conversion utility not available - consider installing the FLAC command line application by running `apt-get install flac` or your operating system's equivalent
